In [1]:
from engine import val_step
from utils.utils import souping, greedy_souping
from models import get_model

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision

from dataset import RSNADataset, HAM10000Dataset, AptosDataset
from torchvision.transforms.v2 import AutoAugmentPolicy, functional as F, InterpolationMode, Transform
from torchvision.transforms import v2
torchvision.disable_beta_transforms_warning()
from timm.data.transforms import RandomResizedCropAndInterpolation
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import argparse

from utils.utils import EarlyStopper, get_dataset
from torch.utils.data.dataset import Subset

import yaml
import json
import time
import os
import wandb
from utils.utils import load_model
import sys


In [12]:
DATASET = 'APTOS'
MODEL = 'DeiT-S'
NUM_CLASSES = 5
DEVICE = 'cuda'
PATH = './dataset/224_data/DR/aptos/""./dataset/224_data/DR/aptos/aptos_dataset_splits.csv'#'/dataset/ISIC/ISIC2018_Task3_Training_Input/""./dataset/ISIC/ISIC2018_Task3_Validation_Input/""./dataset/ISIC/ISIC2018_Task3_Test_Input/""./dataset/ISIC/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv""./dataset/ISIC/ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth.csv""./dataset/ISIC/ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth.csv'
test_sort_by = 'Test F1'
val_sort_by = 'Val F1'
WEIGHT_PATH = f'/l/users/santosh.sanjeev/model_soups/runs/test/fge/{DATASET}/DeiT-S/'


In [13]:
final_val_df = pd.read_csv(os.path.join(WEIGHT_PATH, 'FINAL_VAL_RESULTS.csv'), index_col=[0])
final_test_df = pd.read_csv(os.path.join(WEIGHT_PATH, 'FINAL_TEST_RESULTS.csv'), index_col=[0])

In [14]:
final_val_df

,Model Name,Val Accuracy,Val F1,Val Recall,Val Kappa,Val AUC,LR,model_path
1,25,0.875095,0.774388,0.749560,0.912150,0,0.000001,/l/users/santosh.sanjeev/model_soups/runs/test...
2,10,0.872917,0.774021,0.752154,0.914058,0,0.000005,/l/users/santosh.sanjeev/model_soups/runs/test...
3,14,0.866856,0.769565,0.754965,0.915155,0,0.000005,/l/users/santosh.sanjeev/model_soups/runs/test...
4,1,0.875947,0.768924,0.734147,0.915382,0,0.000010,/l/users/santosh.sanjeev/model_soups/runs/test...
5,19,0.870312,0.766923,0.745258,0.908060,0,0.000010,/l/users/santosh.sanjeev/model_soups/runs/test...
...,...,...,...,...,...,...,...,...
204,12,0.763636,0.512504,0.497879,0.792084,0,0.001000,/l/users/santosh.sanjeev/model_soups/runs/test...
205,4,0.761032,0.511359,0.496768,0.788353,0,0.001000,/l/users/santosh.sanjeev/model_soups/runs/test...
206,1,0.769697,0.508694,0.501579,0.786995,0,0.001000,/l/users/santosh.sanjeev/model_soups/runs/test...
207,0,0.767093,0.506129,0.499579,0.785931,0,0.001000,/l/users/santosh.sanjeev/model_soups/runs/test...


In [15]:
final_test_df

,Model Name,Test Accuracy,Test F1,Test Recall,Test Kappa,Test AUC,LR,model_path
1,23,0.859797,0.741403,0.733067,0.930030,0,1.000000e-04,/l/users/santosh.sanjeev/model_soups/runs/test...
2,25,0.860196,0.740262,0.730923,0.928859,0,1.000000e-04,/l/users/santosh.sanjeev/model_soups/runs/test...
3,8,0.862800,0.739565,0.716779,0.927728,0,1.000000e-04,/l/users/santosh.sanjeev/model_soups/runs/test...
4,22,0.854190,0.735311,0.729067,0.930496,0,1.000000e-04,/l/users/santosh.sanjeev/model_soups/runs/test...
5,21,0.867610,0.734985,0.718609,0.934969,0,1.000000e-04,/l/users/santosh.sanjeev/model_soups/runs/test...
...,...,...,...,...,...,...,...,...
204,2,0.758258,0.523570,0.503588,0.771087,0,1.000000e-03,/l/users/santosh.sanjeev/model_soups/runs/test...
205,4,0.760862,0.521825,0.500921,0.766500,0,1.000000e-03,/l/users/santosh.sanjeev/model_soups/runs/test...
206,3,0.761261,0.520966,0.500921,0.769001,0,1.000000e-03,/l/users/santosh.sanjeev/model_soups/runs/test...
207,0,0.778083,0.518094,0.502076,0.825896,0,1.000000e-07,/l/users/santosh.sanjeev/model_soups/runs/test...


In [2]:

def get_xy(point, origin, vector_x, vector_y):
    return np.array([np.dot(point - origin, vector_x), np.dot(point - origin, vector_y)])


def angle_between_vectors(vectors):
    """
    Calculate the angle between the second and third vectors with respect to the origin.

    Parameters:
    - vectors: numpy array, each row represents a vector

    Returns:
    - angle_deg: float, angle in degrees
    """
    # Extract the second and third vectors from the array
    vector1 = vectors[1]
    vector2 = vectors[2]

    # Calculate vectors from origin to the given points
    origin = np.zeros_like(vector1)
    vector1_from_origin = vector1 - origin
    vector2_from_origin = vector2 - origin

    # Calculate the dot product
    dot_product = np.dot(vector1_from_origin, vector2_from_origin)

    # Calculate the magnitudes
    magnitude_vector1 = np.linalg.norm(vector1_from_origin)
    magnitude_vector2 = np.linalg.norm(vector2_from_origin)

    # Calculate the cosine of the angle
    cosine_theta = dot_product / (magnitude_vector1 * magnitude_vector2)


    # Calculate the angle in radians
    theta_rad = np.arccos(np.clip(cosine_theta, -1.0, 1.0))
    
    # Convert angle to degrees
    angle_deg = np.degrees(theta_rad)

    return angle_deg


In [18]:

best_model_acc = final_val_df.iloc[0][val_sort_by]
best_model_id = 1 #final_val_df.iloc[0]['Model Name']
model_data = []

for index, row in final_val_df.iloc[1:].iterrows():
    # model_id = final_val_df.iloc[index]['Model Name']
    model = get_model(MODEL, num_classes=NUM_CLASSES)
    model = model.to(DEVICE)
    model1 = get_model(MODEL, num_classes=NUM_CLASSES)
    model1 = model1.to(DEVICE)
    model2 = get_model(MODEL, num_classes=NUM_CLASSES)
    model3 = get_model(MODEL, num_classes=NUM_CLASSES)
    model2.load_state_dict(torch.load(final_val_df.iloc[0]['model_path'])['model_state'])
    model3.load_state_dict(torch.load(final_val_df.iloc[index]['model_path'])['model_state'])
    model2 = model2.to(DEVICE)
    model3 = model3.to(DEVICE)

    w = list()
    curve_parameters = [model1.parameters(), model2.parameters(), model3.parameters()]
    for i in range(3):
        w.append(np.concatenate([
            p.data.cpu().numpy().ravel() for p in curve_parameters[i]
        ]))

    u = w[2] - w[0]
    dx = np.linalg.norm(u)
    u /= dx
    v = w[1] - w[0]
    v -= np.dot(u, v) * u
    dy = np.linalg.norm(v)
    v /= dy
    bend_coordinates = np.stack(get_xy(p, w[0], u, v) for p in w)
    angle = angle_between_vectors(bend_coordinates)
    model_data.append((index, angle, final_val_df.iloc[index]['LR']))
    print(index, angle, final_val_df.iloc[index]['LR'])
# Sort the model IDs based on angles in descending order
sorted_model_data = sorted(model_data, key=lambda x: x[1], reverse=True)

# Extract sorted model IDs and angles
sorted_model_ids = [model_id for model_id, angle in sorted_model_data]
sorted_angles = [angle for model_id, angle in sorted_model_data]

sorted_model_ids = [model_id for model_id, angle, LRv in sorted_model_data]
sorted_angles = [angle for model_id, angle, LRv in sorted_model_data]
sorted_LRs = [LRv for model_id, angle, LRv in sorted_model_data]
print("Sorted Model IDs based on angle:")
print(sorted_model_ids)
print("Corresponding Angles:")
print(sorted_angles)
print("Corresponding LR:")
print(sorted_LRs)


2 0.13562092827251518
3 0.14803752532310693
4 0.16432452156180904
5 0.13562092827251518
6 0.034264020096037975
7 0.1282042487280194
8 0.15065786343767226
9 0.16071255014426922
10 0.13988230894202303
11 0.14935344105781187
12 0.13122123336572936
13 0.12194656246489415
14 0.16192549306284887
15 0.15450506866003139
16 0.13270400689029618
17 0.3981127325547928
18 0.1282042487280194
19 0.13122123336572936
20 0.15450506866003139
21 0.12194656246489415
22 0.3891652016442876
23 0.14803752532310693
24 0.11703392759136522
25 0.4044517555675433
26 0.14803752532310693
27 0.4005626901348312
28 0.12354070345060092
29 0.3936643883838593
30 0.39515273375223087
31 0.11534989781745353
32 0.14803752532310693
33 0.423823481996739
34 0.15065786343767226
35 0.4106929493613624
36 0.14803752532310693
37 0.16071255014426922
38 0.17582936122100767
39 0.16071255014426922
40 0.048456642643301504
41 0.3936643883838593
42 0.3921703946518569
43 0.3921703946518569
44 0.15065786343767226
45 0.7375442489242597
46 0.403

IndexError: single positional indexer is out-of-bounds

In [19]:
state_dicts = []
WEIGHT_PATH_v2 = '/l/users/santosh.sanjeev/model_soups/runs/san_final_hyp_models/san-finetune/aptos_final_hyp/deitS_imagenet/2024-01-10_02-53-56/'

for idx, folder in enumerate(sorted(os.listdir(WEIGHT_PATH_v2))):
    print(idx)
    if folder.startswith('testresults'):
        continue
    model_path = os.path.join(WEIGHT_PATH_v2, folder)
    train_summary = json.load(open(os.path.join(model_path, "train_summary.json"), 'r'))
    model_config = train_summary["config"]

    checkpoint = torch.load(os.path.join(model_path, "best_checkpoint.pth"), map_location=DEVICE)
    state_dicts.append(checkpoint['model'])

    model.load_state_dict(checkpoint['model'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [3]:
WEIGHT_PATH_v2 = '/l/users/santosh.sanjeev/model_soups/runs/san_final_hyp_models/san-finetune/aptos_final_hyp/deitS_imagenet/2024-01-10_02-53-56/'
initial_val_df = pd.read_csv('/home/santosh.sanjeev/Projects/model-soups/noodles/test/APTOS/DeiT-S/VAL_RESULTS.csv')
initial_test_df = pd.read_csv('/home/santosh.sanjeev/Projects/model-soups/noodles/test/APTOS/DeiT-S/TEST_RESULTS.csv')
NUM_CLASSES = 5
MODEL = 'DeiT-S'
DEVICE = 'cuda'
val_sort_by = 'Val F1'
initial_model_path = '/home/santosh.sanjeev/Projects/model-soups/noodles/runs/san_final_hyp_models/san-initial_models/aptos_final_hyp/deitS_imagenet/2024-01-10_02-40-01/best_checkpoint.pth'


state_dicts = []
for idx, folder in enumerate(sorted(os.listdir(WEIGHT_PATH_v2))):
    print(idx)
    if folder.startswith('testresults'):
        continue
    model_path = os.path.join(WEIGHT_PATH_v2, folder)
    train_summary = json.load(open(os.path.join(model_path, "train_summary.json"), 'r'))
    model_config = train_summary["config"]

    checkpoint = torch.load(os.path.join(model_path, "best_checkpoint.pth"), map_location=DEVICE)
    state_dicts.append(checkpoint['model'])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [4]:

best_model_acc = initial_val_df.iloc[0][val_sort_by]
best_model_id = initial_val_df.iloc[0]['Model Name']
model_data = []

for index, row in initial_val_df.iloc[1:].iterrows():
    model_id = initial_val_df.iloc[index]['Model Name']
    # model = get_model(MODEL, num_classes=NUM_CLASSES)
    # model = model.to(DEVICE)
    model1 = get_model(MODEL, num_classes=NUM_CLASSES)
    model1.load_state_dict(torch.load(initial_model_path,map_location='cuda')['model'])

    model1 = model1.to(DEVICE)
    model2 = get_model(MODEL, num_classes=NUM_CLASSES)
    model3 = get_model(MODEL, num_classes=NUM_CLASSES)
    model2.load_state_dict(state_dicts[best_model_id])
    model3.load_state_dict(state_dicts[model_id])
    model2 = model2.to(DEVICE)
    model3 = model3.to(DEVICE)

    w = list()
    curve_parameters = [model1.parameters(), model2.parameters(), model3.parameters()]
    for i in range(3):
        w.append(np.concatenate([
            p.data.cpu().numpy().ravel() for p in curve_parameters[i]
        ]))

    u = w[2] - w[0]
    dx = np.linalg.norm(u)
    u /= dx
    v = w[1] - w[0]
    v -= np.dot(u, v) * u
    dy = np.linalg.norm(v)
    v /= dy
    bend_coordinates = np.stack(get_xy(p, w[0], u, v) for p in w)
    angle = angle_between_vectors(bend_coordinates)
    model_data.append((model_id, angle, initial_val_df.iloc[index]['Learning Rate']))
    print(model_id, angle, initial_val_df.iloc[index]['Learning Rate'])
# Sort the model IDs based on angles in descending order
sorted_model_data = sorted(model_data, key=lambda x: x[1], reverse=True)

# Extract sorted model IDs and angles
sorted_model_ids = [model_id for model_id, angle, LRv in sorted_model_data]
sorted_angles = [angle for model_id, angle, LRv in sorted_model_data]
sorted_LRs = [LRv for model_id, angle, LRv in sorted_model_data]
print("Sorted Model IDs based on angle:")
print(sorted_model_ids)
print("Corresponding Angles:")
print(sorted_angles)
print("Corresponding LR:")
print(sorted_LRs)


28 49.885243975275095 1e-05
23 56.93102426170751 5e-05
35 28.547976888693423 5e-06
33 57.506675272865444 5e-06
27 61.44778465962536 1e-05
34 43.52722298685051 5e-06
21 72.52906378256105 5e-05
15 79.80760752829123 0.0001
32 57.2359312872043 5e-06
26 60.83666059327612 1e-05
22 66.8603084164231 5e-05
31 58.853118226846185 5e-06
24 59.37875816465356 1e-05
17 76.20570506099327 0.0001
19 74.35061620864073 5e-05
9 88.42150006320237 0.0005
25 59.26473582355239 1e-05
18 73.97820271184139 5e-05
13 85.74811188459131 0.0001
30 58.50575258020187 5e-06
37 57.73804251163515 1e-06
20 71.95620912039139 5e-05
10 88.72254121373653 0.0005
36 57.57962380552404 1e-06
16 79.16878839180451 0.0001
11 86.54835759356139 0.0005
41 41.46644148210051 1e-06
40 46.28862628692736 1e-06
38 59.754964675876806 1e-06
39 59.98375970214904 1e-06
14 79.9642079044497 0.0001
12 86.2563662851514 0.0001
8 88.8370101054782 0.0005
7 89.13938210577213 0.0005
42 68.08430309960772 1e-07
43 68.12367637517079 1e-07
2 88.8354047865968 0

In [16]:

import copy
train_loader, val_loader, test_loader = get_dataset('APTOS', './dataset/224_data/DR/aptos/""./dataset/224_data/DR/aptos/aptos_dataset_splits.csv', "Minimal", 
                                                        'ImageNet', 224, 128, 16, 'Classification')
CLASSIFICATION = 'MultiClass'
loss = torch.nn.CrossEntropyLoss()
best_model_acc = initial_val_df.iloc[0][val_sort_by]
best_model_id = initial_val_df.iloc[0]['Model Name']
best_ls = [best_model_id]
model_data = []
ls = sorted_model_ids#[33, 30, 34, 37, 32, 31, 38, 41, 36, 35, 39, 21, 18, 22, 25, 20, 19, 26, 29, 23, 24, 40, 0, 3, 27, 12, 42, 45, 14, 13, 16, 15, 17, 7, 2, 4, 1, 44, 49, 46, 43, 50, 53, 47, 48, 8, 11, 51, 5, 6, 52, 9, 10]
dkfjsdfk = 0
while True:
    if dkfjsdfk >= len(ls):
        break
    model_id = ls[dkfjsdfk]
    print(model_id)
    model = get_model(MODEL, num_classes=NUM_CLASSES)
    model = model.to(DEVICE)
    model1 = get_model(MODEL, num_classes=NUM_CLASSES)
    model1.load_state_dict(torch.load(initial_model_path,map_location='cuda')['model'])
    model1 = model1.to(DEVICE)
    model2 = get_model(MODEL, num_classes=NUM_CLASSES)
    model3 = get_model(MODEL, num_classes=NUM_CLASSES)
    model2.load_state_dict(state_dicts[best_model_id])
    model3.load_state_dict(state_dicts[model_id])
    model2 = model2.to(DEVICE)
    model3 = model3.to(DEVICE)

    sd = [state_dicts[best_model_id], state_dicts[model_id]]
    alphal = [len(best_ls)/(len(best_ls)+1),1- (len(best_ls)/(len(best_ls)+1))]
    print(alphal)
    # alphal = [0.5,0.5]
    uniform_model = souping(model, sd, alphal)
    uniform_test_loss, uniform_test_acc, uniform_test_f1, uniform_test_recall, uniform_test_kappa, uniform_test_auc = val_step(uniform_model, val_loader, train_loader, loss, DEVICE, CLASSIFICATION)
    print(uniform_test_f1, best_model_acc)
    if uniform_test_f1 > best_model_acc:
        print('in uniform testtttttttttttttttt')
        best_model_acc = uniform_test_f1
        state_dicts[best_model_id] = copy.deepcopy(uniform_model.state_dict())
        best_ls.append(model_id)
        print(uniform_test_f1, best_ls)
        ls.remove(model_id)
        print(ls)#ls[ls.index(model_id) + 1:]
        model2 = get_model(MODEL, num_classes=NUM_CLASSES)
        model3 = get_model(MODEL, num_classes=NUM_CLASSES)

        model2.load_state_dict(state_dicts[best_model_id])
        model2 = model2.to(DEVICE)
        angles_with_remaining_models = []
        for j in ls:
            model_j_id = j
            model3 = get_model(MODEL, num_classes=NUM_CLASSES)
            model3.load_state_dict(state_dicts[model_j_id])
            model3 = model3.to(DEVICE)


            w = list()
            curve_parameters = [model1.parameters(), model2.parameters(), model3.parameters()]
            for i in range(3):
                w.append(np.concatenate([
                    p.data.cpu().numpy().ravel() for p in curve_parameters[i]
                ]))

            u = w[2] - w[0]
            dx = np.linalg.norm(u)
            u /= dx
            v = w[1] - w[0]
            v -= np.dot(u, v) * u
            dy = np.linalg.norm(v)
            v /= dy
            bend_coordinates = np.stack(get_xy(p, w[0], u, v) for p in w)
            angle = angle_between_vectors(bend_coordinates)
            angles_with_remaining_models.append((model_j_id, angle))

        # Sort ls based on angles in descending order
        ls = [model_id for model_id, _ in sorted(angles_with_remaining_models, key=lambda x: x[1], reverse=True)]

        # Print the sorted ls and corresponding angles
        print("Sorted Model IDs based on angle:")
        print(ls)
        print("Corresponding Angles:")
        print([angle for _, angle in sorted(angles_with_remaining_models, key=lambda x: x[1], reverse=True)])
        dkfjsdfk = 0
    else:
        dkfjsdfk+=1


['./dataset/224_data/DR/aptos/', './dataset/224_data/DR/aptos/aptos_dataset_splits.csv']
APTOS 2929 366 367
6
[0.5, 0.5]


100%|██████████| 3/3 [00:05<00:00,  1.74s/it]


0.3044290712468193 0.7619860919530116
0
[0.5, 0.5]


100%|██████████| 3/3 [00:05<00:00,  1.77s/it]


0.2871482248293843 0.7619860919530116
7
[0.5, 0.5]


...

0.7439011128740696 0.7855317939457205
32
[0.6666666666666666, 0.33333333333333337]


100%|██████████| 3/3 [00:04<00:00,  1.64s/it]


0.7601903614695449 0.7855317939457205
33
[0.6666666666666666, 0.33333333333333337]


100%|██████████| 3/3 [00:05<00:00,  1.68s/it]

0.7591136139737015 0.7855317939457205


In [15]:

model = get_model(MODEL, num_classes=NUM_CLASSES)
model = model.to(DEVICE)
sd = []
best_ls = [29, 28]#, 20]#, 27]
for i in best_ls:
    sd.append(state_dicts[i])
alphal = [1/len(best_ls)]*len(best_ls)
# alphal = [0.25,0.25, 0.25,0.25]
print(best_ls)
print(alphal)
uniform_model = souping(model, sd, alphal)
uniform_test_loss, uniform_test_acc, uniform_test_f1, uniform_test_recall, uniform_test_kappa, uniform_test_auc = val_step(uniform_model, val_loader, train_loader, loss, DEVICE, CLASSIFICATION)
print(uniform_test_acc, uniform_test_f1, uniform_test_recall, uniform_test_kappa, uniform_test_auc)

uniform_test_loss, uniform_test_acc, uniform_test_f1, uniform_test_recall, uniform_test_kappa, uniform_test_auc = val_step(uniform_model, test_loader, train_loader, loss, DEVICE, CLASSIFICATION)

print(uniform_test_acc, uniform_test_f1, uniform_test_recall, uniform_test_kappa, uniform_test_auc)


[29, 28]
[0.5, 0.5]


100%|██████████| 3/3 [00:05<00:00,  1.90s/it]


0.8733428030303031 0.768060500157213 0.7448777052914983 0.9077142653250553 0


100%|██████████| 3/3 [00:05<00:00,  1.77s/it]

0.8407704579579579 0.6919765706491214 0.682011013048524 0.9106395716382336 0
